# El Camino hacia el Óptimo: Construyendo el Algoritmo

## Método de Máximo Descenso
Bien... el algoritmo es bastante sencillo e intuitivo, si bien está en Python  
se entiende casi como si fuera pseudocódigo. La base de nuestro desarrollo  
comienza con:

In [ ]:
import numpy as np

### Implementación en Python del Algoritmo de Máximo Descenso Óptimo:

In [ ]:
def maximo_descenso_optimo(Q, c, x0, tol=1e-8, max_iter=1000, verbose=False):
    """
    Máximo descenso para f(x) = 1/2 x^T Q x + c^T x + r
    usando el paso analítico óptimo alpha_k = (g^T g) / (g^T Q g).
    NOTA: La expresión alpha_k fue previamente hallada de forma analítica (en el LaTeX).

    Parámetros:
        Q : ndarray (n,n)  -- matriz simétrica
        c : ndarray (n,)   -- vector
        x0: ndarray (n,)   -- punto inicial
        tol: float         -- tolerancia para ||gradiente||
        max_iter: int      -- número máximo de iteraciones
        verbose: bool      -- imprimir o no el progreso

    Retorna:
        x : ndarray (n,)   -- punto mínimo (x1, ..., xn) en nuestro modelo x = (x1, x2)
        info : dict        -- historial (opcional)
    """
    # --- Validaciones simples ---
    Q = np.array(Q, dtype=float)
    c = np.array(c, dtype=float)
    x = np.array(x0, dtype=float)

    # --- Dimensiones ---
    n = x.shape[0]
    assert Q.shape == (n, n)
    assert c.shape == (n,)

    # --- Comprobar simetría numérica ---
    if not np.allclose(Q, Q.T, atol=1e-12):
        Q = 0.5 * (Q + Q.T)

    # Historial para análisis
    iters = []
    fvals = []
    alphas = []
    norms = []
    xs = []  # almacenará el vector x de cada iteración (incluye inicio y fin)

    # Guardar x inicial
    xs.append(x.copy())

    # --- Función objetivo ---
    def f_val(x_vec):
        return 0.5 * x_vec @ (Q @ x_vec) + c @ x_vec

    # --- Iteraciones ---
    for k in range(1, max_iter + 1):
        g = Q @ x + c                      # gradiente g_k = Q x_k + c
        gn = np.linalg.norm(g)             # ||g_k||

        iters.append(k)
        fvals.append(f_val(x))
        norms.append(gn)

        if verbose:
            print(f"Iter {k:3d}: ||g|| = {gn:.3e}, f = {fvals[-1]:.6e}")

        # Criterio de parada
        if gn < tol:
            if verbose:
                print("Criterio de parada alcanzado (||g|| < tol).")
            # Guardar x final antes de salir (si se cumple criterio de parada antes de actualizar)
            xs.append(x.copy())
            break

        Qg = Q @ g                         # producto Q g_k (reutilizable)
        denom = g @ Qg                     # g^T Q g

        # Seguridad numérica: evitar división por cero / denominador no positivo
        if denom <= 0:
            raise ValueError(
                "Denominador g^T Q g ≤ 0. Q puede no ser definida positiva o g≈0."
            )

        alpha = (g @ g) / denom            # alpha_k óptimo
        alphas.append(alpha)

        # Actualización
        x = x - alpha * g
        xs.append(x.copy())

    # Guardar x después de la actualización
    xs.append(x.copy())

    info = {
        "iters": np.array(iters),
        "fvals": np.array(fvals),
        "alphas": np.array(alphas),
        "norms": np.array(norms),
    }
    # Añadir trayectorias de x (cada fila corresponde a una iteración, incluyendo inicial y final)
    info["xs"] = np.vstack(xs)

    # Redondear cada elemento de x (evitar errores numéricos)
    x = np.round(x, decimals=2)

    return x, info

### Parámetros y flujo del algoritmo de Máximo Descenso Óptimo


El enfoque de esta implementación se fundamenta en la expresión cuadrática de nuestro modelo y toma como entrada los siguientes parámetros:


- `Q`: Matriz cuadrada y simétrica que representa los coeficientes de la función cuadrática.
- `c`: Vector que acompaña a la matriz `Q` en la función objetivo. Junto con `Q`, define la forma de la función a minimizar.
- `x0`: Punto inicial desde donde comienza el algoritmo; es la primera estimación de la solución.
- `tol`: Tolerancia que determina cuándo detener el algoritmo. Si la norma del gradiente es menor que este valor, se asume que se alcanzó una solución suficientemente buena.
- `max_iter`: Número máximo de iteraciones permitidas, para evitar que el algoritmo se ejecute indefinidamente si no converge.
- `verbose`: Si es `True`, imprime información del progreso en cada iteración. Útil para depuración, seguimiento y futuras comparaciones con otros algoritmos.


#### Validación y verificación de datos (líneas 19–31)


- Líneas 19–22: Convierte las entradas `Q`, `c` y `x0` a arreglos de NumPy de tipo `float`. Esto garantiza un formato numérico uniforme y evita errores de tipo, truncamiento o redondeo.
- Líneas 24–27: Calcula la dimensión del problema (`n`) y verifica que `Q` sea de tamaño `n×n` y que `c` tenga tamaño `n`. Así se evitan fallos por incompatibilidades de dimensiones.
- Líneas 29–31: Verifica si `Q` es simétrica (dentro de una tolerancia numérica). Si no lo es, se fuerza la simetría promediándola con su traspuesta: $Q \leftarrow \tfrac{1}{2}(Q + Q^\top)$.

  Por ejemplo, la matriz:

  \begin{bmatrix}
    10 & 6,000001 \\
  5,999997 & 4
  \end{bmatrix}

  es teóricamente simétrica, pero por errores numérico-computacionales puede no ser reconocida exactamente como tal.


#### Historial del proceso (líneas 33–38)


Se inicializan listas vacías para registrar el historial de la optimización:


- `iters`: Número de iteración (1, 2, 3, ...).
- `fvals`: Valor de la función objetivo $f(x)$ en cada paso.
- `alphas`: Tamaño de paso óptimo $\alpha_k$ calculado en cada iteración.
- `norms`: Norma del gradiente $\lVert g_k \rVert$, usada como criterio de convergencia.


#### Función objetivo (líneas 43–45)


Se define la función objetivo cuadrática $f(x) = \tfrac{1}{2} x^\top Q x + c^\top x$.


Nota: el término constante $r$ se omite porque no afecta el gradiente ni, por ende, la optimización.


#### Bucle de iteraciones (líneas 47–81)


Se ejecuta hasta alcanzar la tolerancia (en norma del gradiente) o el número máximo de iteraciones:


- Línea 48: Itera a lo sumo `max_iter`.


- Línea 49: Calcula el gradiente en el punto actual: $g_k = Q x_k + c$.


- Línea 50: Calcula la norma euclidiana del gradiente $\lVert g_k \rVert$.


- Líneas 52–54: Registra el número de iteración, el valor de $f(x)$ y la norma del gradiente para análisis posterior.


- Líneas 56–57: Si `verbose` está activo, imprime la información de la iteración (norma del gradiente y valor de la función).


- Líneas 59–65: Criterio de parada: si $\lVert g_k \rVert < \text{tol}$, se asume que se alcanzó un mínimo y se detiene el algoritmo.


- Líneas 67–68: Calcula $Q g_k$ y el escalar $g_k^\top Q g_k$, necesarios para el paso óptimo.


- Líneas 70–74: Si el denominador $g_k^\top Q g_k \le 0$, se lanza un error (el paso no sería válido). Esto aporta seguridad numérica.


- Líneas 76–77: Tamaño de paso óptimo: $\alpha_k = \dfrac{g_k^\top g_k}{g_k^\top Q g_k}$. Luego, se guarda para análisis posterior.


- Líneas 79–81: Actualización: $x_{k+1} = x_k - \alpha_k g_k$.


#### Salida y postprocesado (líneas 83–91 y 92–98)


- Se construye el diccionario `info` con el historial: `iters`, `fvals`, `alphas`, `norms`.


- Se redondea cada componente de la solución `x` (p. ej., a 2 decimales) para evitar mostrar artefactos numéricos.


- Finalmente, se retorna el par `(x, info)`.

## Método de Newton
A partir del resultado obtenido en en LaTeX, se ha logrado desarrollar un algoritmo carac
terizado por su simplicidad estructural:

### Implementación del método de Newton para problemas de naturaleza cuadrática, como nuestro modelo:

In [ ]:
def newton_quadratic(Q, c, x0=None):
    """
    Método de Newton para función cuadrática
    f(x) = 1/2 x^T Q x + c^T x + r.

    Para este caso, una sola iteración produce el óptimo:
        x* = -Q^{-1} c

    Parámetros
    - Q: ndarray (n, n), matriz simétrica e invertible.
    - c: ndarray (n,), vector.
    - x0: ndarray (n,), punto inicial opcional (no afecta el resultado final en el caso cuadrático).

    Retorna
    - x1: ndarray (n,), punto óptimo (en una iteración, x1 = -Q^{-1} c).
    - info: ndarray (2, n), arreglo con los puntos [x0, x1] para formar una trayectoria mínima.
    """
    n = Q.shape[0]
    if x0 is None:
        x0 = np.zeros(n)
    b = - (Q @ x0 + c)        # -grad f(x0)
    # Resolver Q p = b  (p = paso de Newton)
    p = np.linalg.solve(Q, b)
    x1 = x0 + p               # en una iteración, x1 = -Q^{-1} c
    info = np.vstack([x0, x1])  # trayectoria: inicio y final
    return x1, info

Veamos cada aspecto:



- **Línea 1:** Se define la función que recibe la matriz $Q$, el vector $c$ y, opcionalmente, un punto inicial $x_0$. Si no se proporciona $x_0$, se utilizará el vector cero.



- **Líneas 18-20:** Se obtiene la dimensión del problema con `n = Q.shape[0]`. Si $x_0$ no fue dado, se crea un vector nulo del tamaño adecuado: `x0 = np.zeros(n)`. Esto garantiza coherencia dimensional entre $Q$, $c$ y $x_0$.



- **Línea 21:** Se calcula el gradiente en $x_0$:

  

  $$\nabla f(x_0) = Q x_0 + c$$

  

  Luego se define $b = -(Q x_0 + c)$, como el negativo del gradiente, que representa la dirección de descenso de Newton.



- **Línea 23:** Se resuelve el sistema lineal:

  

  $$Q p = b$$

  

  mediante `np.linalg.solve(Q, b)`. Este paso calcula el vector de corrección $p$ sin invertir explícitamente $Q$, lo que es más estable y eficiente.



- **Líneas 24-26:** Se actualiza el punto con:

  

  $$x_1 = x_0 + p$$

  

  Es decir, se avanza una iteración de Newton. En una función cuadrática, este único paso da directamente el mínimo:

  

  $$x^* = -Q^{-1} c$$

  y finalmente, se devuelve $x_1$, el nuevo punto (el mínimo).

  

##  Representación Gráfica del Modelo

In [ ]:
from __future__ import annotations
from typing import Callable, Optional
import numpy as np
import matplotlib.pyplot as plt


def graficar_superficie(
        f: Callable[[np.ndarray, np.ndarray], np.ndarray],
        xlim: tuple[float, float] = (-3.0, 3.0),
        ylim: tuple[float, float] = (-3.0, 3.0),
        n: int = 100,
        xlabel: str = 'x', ylabel: str = 'y', zlabel: str = 'z', title: str = '',
        cmap: str = 'viridis', elev: Optional[float] = None, azim: Optional[float] = None,
        dist: Optional[float] = None, expand: float = 1.0
    ) -> None:
    """
    Grafica la superficie z = f(x, y) sin mostrar plano de corte visible.
    Muestra también el contorno proyectado sobre el plano XY (como en el ejemplo sin(R)/R).
    """
    # Ajuste del rango
    expand = float(expand)
    if expand < 1.0:
        expand = 1.0
    if expand != 1.0:
        cx = 0.5 * (xlim[0] + xlim[1])
        hx = 0.5 * (xlim[1] - xlim[0]) * expand
        xlim = (cx - hx, cx + hx)
        cy = 0.5 * (ylim[0] + ylim[1])
        hy = 0.5 * (ylim[1] - ylim[0]) * expand
        ylim = (cy - hy, cy + hy)

    # Malla
    x = np.linspace(*xlim, int(n))
    y = np.linspace(*ylim, int(n))
    X, Y = np.meshgrid(x, y)
    Z = np.asarray(f(X, Y), dtype=float)

    # Recorte invisible (los valores que superan 15000 se hacen NaN)
    Z[Z > 15000] = np.nan

    # Figura
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Superficie principal
    surf = ax.plot_surface(X, Y, Z, cmap=cmap, linewidth=0, antialiased=True, alpha=0.95)

    # Contorno proyectado sobre el plano XY (sin mostrar plano)
    ax.contour(X, Y, Z, zdir='z', offset=np.nanmin(Z) - 500, cmap=cmap, linewidths=1.5)

    # Etiquetas
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_zlabel(zlabel)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_zlim(np.nanmin(Z) - 500, 15000)  # no muestra corte visible

    if title:
        ax.set_title(title)

    # Ajuste de cámara
    if elev is not None or azim is not None:
        ax.view_init(elev=elev or 30, azim=azim or -60)
    if dist is not None and hasattr(ax, 'dist'):
        try:
            ax.dist = float(dist)
        except Exception:
            pass

    fig.colorbar(surf, shrink=0.65, aspect=12, pad=0.1)
    plt.tight_layout()
    plt.show()

La función está diseñada para generar una visualización 3D de una función de dos variables $f(x,y)$. Produce tanto la superficie principal como un contorno proyectado sobre el plano XY, con control sobre rangos, resolución, apariencia y perspectiva de la cámara. Está optimizada para manejar valores extremos de la función sin que afecten la visualización.


### Parámetros y configuración de la malla:

- `xlim` y `ylim` definen los rangos de los ejes X e Y; se pueden expandir mediante `expand` para aumentar el área visible.

- `n` indica el número de puntos en la malla para cada eje; mayor `n` produce una malla más fina y superficie más suave.

- Se genera una cuadrícula 2D `X`, `Y` usando `np.meshgrid` y se evalúa `f` sobre esta malla, produciendo `Z`.

- Los valores de `Z` que exceden 15,000 se reemplazan con `NaN` para evitar cortes visibles abruptos en la gráfica.


### Visualización 3D:

- Se crea una figura y un eje 3D con `matplotlib`.

- La superficie principal se grafica con `plot_surface`, aplicando un colormap (`cmap`) y transparencia (`alpha`).

- Se añade un contorno proyectado en el plano XY usando `ax.contour`, para visualizar la topografía de la función incluso sin mostrar el plano de base.


### Ajustes de etiquetas y límites:

- `xlabel`, `ylabel` y `zlabel` etiquetan los ejes.

- Los límites de los ejes se ajustan según los rangos y la expansión.

- El eje Z se limita desde un mínimo basado en `Z` hasta 15,000, asegurando que no aparezcan cortes visibles de valores altos.


### Control de cámara y estética:

- `elev` y `azim` permiten ajustar la elevación y azimut de la cámara para cambiar el ángulo de vista.

- `dist` modifica la distancia de la cámara respecto a la figura si el atributo existe.

- Se añade una barra de color asociada a la superficie para representar magnitudes de `Z` y se ajusta el layout con `tight_layout` para evitar solapamientos.